# Pickling and FastText models

## Setup

Here, we will import the necessary modules and define the convenience functions we will use throughout this example

In [ ]:
import pathlib
import pickle
import tempfile

from typing import Self, Any

import numpy as np

from lorem_text import lorem
from fasttext import FastText
from sklearn import pipeline, base, cluster


def make_fake_data(n_samples: int = 100) -> list[str]:
    return [lorem.sentence() for _ in range(n_samples)]


def save_pipeline(pipeline: pipeline.Pipeline, filename: str = "saved_pipeline.pkl") -> None:
    with open(filename, "wb") as f:
        pickle.dump(pipeline, f)


def load_pipeline(filename: str = "saved_pipeline.pkl") -> pipeline.Pipeline:
    with open(filename, "rb") as f:
        return pickle.load(f)


def train_model(corpus: list[str]) -> FastText._FastText:
    with tempfile.TemporaryDirectory() as d:
        data_file = pathlib.Path(d) / "training_data.txt"
        data_file.write_text("\n".join(corpus) + "\n")
        model = FastText.train_unsupervised(str(data_file))
    return model

## The problem

For the purposes of illustration, we are going to be making a clustering
[Pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators)
with two steps

  1. a [FastText](https://fasttext.cc/) embedding transformer step
  2. a [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) step

We are then going to try to serliaize this `Pipeline` using Python's
[pickle](https://docs.python.org/3/library/pickle.html) module.

### Define the `FastTextTransformer`

This is done by implementing the following:

  1. inherit from [base.BaseEstimator](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html#sklearn-base-baseestimator) and [base.TransformerMixin](https://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html#sklearn-base-transformermixin)
  2. implement both `fit` and `transform` methods

In [ ]:
class FastTextTransformer(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, model: FastText._FastText):
        self.model = model
        super().__init__()

    def fit(self, *args: Any, **kwargs: Any) -> Self:
        # No-Op
        return self

    def transform(self, X: np.ndarray) -> np.ndarray:
        text = np.atleast_1d(X)

        if text.ndim != 1:
            raise ValueError(f"`X` must be 1-dimensional, received {text.ndim}d data")

        return np.asarray([self.model.get_sentence_vector(s) for s in text.tolist()])

### Train your FastText model and construct your clustering pipeline

#### Train the model

In [ ]:
fasttext_trainig_data = make_fake_data()
fasttext_model = train_model(corpus=fasttext_trainig_data)

#### Train your cluster model

In [ ]:
cluster_training_data = make_fake_data()
random_state = np.random.RandomState(1024)  # setting random state for comparability

unpicklable_pipeline = pipeline.make_pipeline(
    FastTextTransformer(model=fasttext_model), cluster.KMeans(n_clusters=2, n_init="auto", random_state=random_state)
).fit(cluster_training_data)

#### Save your pipeline

Here we should get
> `TypeError: cannot pickle 'fasttext_pybind.fasttext' object`

In [ ]:
save_pipeline(unpicklable_pipeline)

### What happened?

`fasttext_pybind.fasttext` is a compiled C++ extension and so cannot be serialized cross-platform.

### Are we out of luck?

At this point we have two choices:

  1. accept that we can't use `FastText` models in our pipeline transformations
  2. learn some pickling dark magic, and make an object that we can pickle

Option $1$ is no fun and wouldn't make for a very interesting post so let's go with option $2$.

### What can we do?

Fortunately, we have a couple things going for us.

  1. `fasttext` provides its own `load_model` and `save_model` methods (see `model` [docs](https://fasttext.cc/docs/en/python-module.html#model-object))
  2. Object pickling in Python, like most things, relies on a protocol that we are free to hack away at# %% [markdown]

### The solution

With this in mind, we simply wrap `fasttext.load_model` and
`FastText._FastText.save_model` functionality with Python's pickling
protocol. This requires us to extend `FastText._FastText` to include:

1. a method to serialize a trained model to `bytes`
3. a constructor to instantiate a model from its serialized `bytes` representatino
2. override the [\_\_reduce\_\_](https://docs.python.org/3/library/pickle.html#object.__reduce__) method to return a `tuple` containing
  > - A callable object that will be called to create the initial version of the object.
  > - A tuple of arguments for the callable object. An empty tuple must be given if the callable does not accept any argument.

In our case `__reduce__` will return a `tuple` containing the constructor in
(2) and a `tuple` containing the `bytes` of our serialized instance returned
from the method in (1).

### Putting it all together

Finally, we land on the following implementation

In [ ]:
import pathlib
import tempfile
from typing import Callable, Self

import fasttext
from fasttext import FastText


class PicklableFastText(FastText._FastText):  # type: ignore[no-any-unimported]
    _tmp_filename = "model.bin"

    @classmethod
    def from_pretrained(cls, model: FastText._FastText) -> Self:  # type: ignore[no-any-unimported]
        self = cls.__new__(cls)
        self.__dict__.update(model.__dict__)
        return self

    @classmethod
    def load(cls, saved_model: bytes | str) -> Self:
        if isinstance(saved_model, str) and pathlib.Path(saved_model).exists():
            return cls.from_pretrained(fasttext.load_model(saved_model))

        if isinstance(saved_model, bytes):
            with tempfile.TemporaryDirectory() as d:
                model_path = pathlib.Path(d) / cls._tmp_filename
                model_path.write_bytes(saved_model)
                return cls.load(str(model_path))

        raise FileNotFoundError(saved_model)

    def serialize(self) -> bytes:
        with tempfile.TemporaryDirectory() as d:
            model_path = pathlib.Path(d) / self._tmp_filename
            self.save_model(str(model_path))
            return model_path.read_bytes()

    def __reduce__(self) -> tuple[Callable[[bytes | str], Self], tuple[bytes]]:
        return self.load, (self.serialize(),)

#### Update our original pipeline with our `PicklableFastText` model.

In [ ]:
picklable_pipeline = pipeline.make_pipeline(
    FastTextTransformer(model=PicklableFastText.from_pretrained(fasttext_model)),
    cluster.KMeans(n_clusters=2, n_init="auto", random_state=random_state),
).fit(cluster_training_data)

#### Save our new Pipeline

In [ ]:
save_pipeline(picklable_pipeline)

#### Does it work?

In [ ]:
sentence = lorem.sentence()

print(picklable_pipeline.transform([sentence]))
print(load_pipeline().transform([sentence]))